In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,  GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier

s3 = boto3.resource('s3')
bucket_name = "rachaeld-data445"
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
# removing observations with NA
heart = heart.dropna()

In [ ]:
RFRecall = list()
RFAccuracy = list()
ADARecall = list()
ADAAccuracy = list()
GBRecall = list()
GBAccuracy = list()

for i in range (0,100):
    X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate']]
    Y = heart['TenYearCHD']
    #splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = .2)

## Model 1- Random Forest    ### with 500 trees and max tree depth equal 3 ###
    #building the model
    RF_md = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)
    #predicting on test
    RF_pred = RF_md.predict_proba(X_test)[:, 1]
    #changing liklihoods to labels
    RF_label = np.where(RF_pred < .1, 0, 1)

    #computing the recall/accuracy score
    RF_recall = recall_score(Y_test, RF_label)
    RF_accuracy = accuracy_score(Y_test, RF_label)
    RFRecall.append(RF_recall)
    RFAccuracy.append(RF_accuracy)

## Model 2- Adaboost ### with 500 trees, max tree depth equal 3, and learning rate equal to 0.01 ###
    #building the model 
    ADA_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = .01).fit(X_train, Y_train)
    #predicting on test
    ADA_pred = ADA_md.predict_proba(X_test)[:,1]
    # changing liklihoods to labels
    ADA_label = np.where(ADA_pred < .1, 0,1)

    #computing the recall/accuracy score
    ADA_recall = recall_score(Y_test, ADA_label)
    ADARecall.append(ADA_recall)
    ADA_accuracy = accuracy_score(Y_test, ADA_label)
    ADAAccuracy.append(ADA_accuracy)

## Model 3- Gradient boost  ### with 500 trees, max tree depth equal 3, and learning rate equal to 0.01 ###
    #building the model 
    GB_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = .01).fit(X_train, Y_train)
    #predicting on test
    GB_pred = GB_md.predict_proba(X_test)[:,1]
    # changing liklihoods to labels
    GB_labels = np.where(GB_pred < .1, 0,1)

    #computing the recall/accuracy score
    GB_recall = recall_score(Y_test, GB_labels)
    GBRecall.append(ADA_recall)
    GB_accuracy = accuracy_score(Y_test, GB_labels)
    GBAccuracy.append(GB_accuracy)
    


print('The recall score of the random forest model is:', np.mean(RFRecall))
print('The accuracy score of the random forest model is:', np.mean(RFAccuracy))

print('The recall score of the ADABoost model is:', np.mean(ADARecall))
print('The accuracy score of the ADABoost model is:', np.mean(ADAAccuracy))

print('The recall score of the Gradient Boost model is:', np.mean(GBRecall))
print('The accuracy score of the Gradient Boost model is:', np.mean(GBAccuracy))